# API Github

In [ ]:
import requests 

In [ ]:
r = requests.get('https://api.github.com/events')

In [ ]:
r 

In [ ]:
r = requests.get('https://api.github.com/versions')
r.status_code

In [ ]:
r.json()

# Extract

In [ ]:
# especificando versão da API 
headers = {'X-GitHub-Api-Versions':'2022-11-28'}

In [ ]:
api_base_url = 'https://api.github.com'
owner = 'amzn' # user para extrair os dados
url = f'{api_base_url}/users/{owner}/repos'

In [ ]:
url

In [ ]:
response = requests.get(url, headers=headers)
response.status_code

In [ ]:
len(response.json())

## autorização

In [ ]:
import os
access_token = os.getenv('TOKEN_GITHUB')
headers = {'Authorization': f'Bearer {access_token}','X-GitHub-Api-Versions':'2022-11-28'}

## paginação

In [ ]:
#lista de paginas + .json()
repos_list = [] # lista composta por sub listas.
for page_num in range(1,6):
    params = {'page': page_num,'per_page': 100} # de 100 em 100 ate a 5 page.
    try:
        response = requests.get(url,headers=headers,params=params)
        if response.status_code == 200:
            repos_list.append(response.json())
        else:
            print(f'Página {page_num} retornou status {response.status_code}')
            repos_list.append(None)
    except Exception as e:
        print(f'Erro ao acessar página {page_num}: {e}')
        repos_list.append(None)

In [ ]:
len(repos_list) # paginas

In [ ]:
len(repos_list[0]) # conteudo

# Transform

In [ ]:
repos_list[0][4]['name'] # abre uma das listas e checa dados(name)

In [ ]:
repos_name = []
for page in repos_list: # lista com varias listas
    if page is not None:
        for repo in page: # dentro de uma das listas
            repos_name.append(repo['name']) # adiciona o nome dentro de cada uma das listas da lista de listas.

In [ ]:
len(repos_name) # checando se pegou todos os repositorios da API.

In [ ]:
repos_name[:10]

In [ ]:
repos_lang = []
for page in repos_list:
    if page is not None:
        for repos in page:
            repos_lang.append(repos['language'])

In [ ]:
repos_lang[:5]

In [ ]:
len(repos_lang)

## DataFrame

In [ ]:
import pandas as pd

In [ ]:
dados_amz = pd.DataFrame() # fazendo o DF
dados_amz['repository Name'] = repos_name # criando coluna para lista de names.
dados_amz['Languages'] = repos_lang # coluna de linguagens usadas.

In [ ]:
# checagem 
dados_amz

In [ ]:
dados_amz.to_csv('amazon.csv')

# Load

In [ ]:
api_base_url = 'https://api.github.com' # api de antes
url = f'{api_base_url}/user/repos'
url

In [ ]:
# criando repositorio POST
data = {
    'name':'linguagens_utilizadas',
    'description':'Repositorio com linguages de Prog Amazon',
    'private': False # true == privado | false == publico

response = requests.post(url, json=data, headers=headers) # esse codigo vai criar o repositorio com as caractericas de data
response.status_code

In [ ]:
# colocando o arquivo no formato base64
import base64 

In [ ]:
with open('amazon.csv', 'rb') as file: # lê o arquivo csv em binario
    file_content = file.read()

encoded_content = base64.b64encode(file_content) #arquivo base64 da DF na variavel enconded_content

In [ ]:
#subindo o arquivo para o repositorio
api_base_url = 'https://api.github.com'
username = 'FireFox-exe'
repo = 'linguagens_utilizadas'
path = 'amazon.csv'

url = f'{api_base_url}/repos/{username}/{repo}/contents/{path}'
url

In [ ]:
# PUT
data = {
    'message': 'adicionando novo arquivo',
    'content': encoded_content.decode('utf-8') # de base4 para texto
}

response = requests.put(url, json=data, headers=headers)
response.status_code